# Matrix Product Operator of Square Lattice Compass Model

Author: Haimeng Zhao

Email: haimengzhao@icloud.com

This notebook investigate the matrix product operator (MPO) form of the Hamiltonian of a compass model.

We first write down the finite automata form of the Hamiltonian, and from that obtain the matrix product operator.

Then we contract the MPOs to generate the Hamiltonian, diagonalize it, and compare the results with exact diagonalization.

## Compass Model
The model we are interested here is on a $3\times 3$ square lattice, with periodic boundary condition in vertical direction and open boundary condition in horizontal direction:

<img src="./model.png" width="10%"/>

The Hamiltonian is of the following form:
$$
H = -J_x\sum_{(i, j)_H}\sigma_i^x\sigma_j^x - J_y\sum_{(i, j)_V}\sigma_i^y\sigma_j^y,
$$
where $J_x=0.4, J_y=1.3$ and
$$
(i, j)_H = (0,3), (3,6), (1,4), (4,7), (2,5), (5,8)
$$
$$
(i, j)_V = (0,1), (1,2), (2,0), (3,4), (4,5), (5,3), (6,7), (7,8), (8,6)
$$

## Finite Automata Diagram

Our goal is to find the MPO representation of the above Hamiltonian, i.e.

<img src="./MPO.png" width="50%"/>

A proper finite automata diagram would help us a lot.

From the form of the Hamiltonian, we can write down the automata diagram of the vertical links:

<img src="./automata-sigmay.png" width="50%"/>

and the horizontal links:

<img src="./automata-sigmax.png" width="50%"/>

Adding these two diagram, we get:

<img src="./automata.png" width="50%"/>


## Matrix Product Operator
The diagram above has 6 rows and a period of 3. 

Therefore, we only need 3 different $6\times 6$ matrix operator component, which can be read off directly from the diagram (assuming $J_x=1, J_y=1$):

$$
M_0 = \begin{pmatrix}
I &0 &0 &\sigma^x &\sigma^y &0\\
0 &I &0 &0 &0 &0\\
0 &0 &I &0 &0 &0\\
0 &0 &0 &0 &0 &\sigma^x\\
0 &0 &0 &0 &0 &0\\
0 &0 &0 &0 &0 &I
\end{pmatrix}
$$

$$
M_1 = \begin{pmatrix}
I &0 &\sigma^x &0 &\sigma^y &0\\
0 &I &0 &0 &0 &0\\
0 &0 &0 &0 &0 &\sigma^x\\
0 &0 &0 &I &0 &0\\
0 &0 &0 &0 &I &\sigma^y\\
0 &0 &0 &0 &0 &I
\end{pmatrix}
$$

$$
M_2 = \begin{pmatrix}
I &\sigma^x &0 &0 &0 &0\\
0 &0 &0 &0 &0 &\sigma^x\\
0 &0 &I &0 &0 &0\\
0 &0 &0 &I &0 &0\\
0 &0 &0 &0 &0 &\sigma^y\\
0 &0 &0 &0 &0 &I
\end{pmatrix}
$$

And the Hamiltonian is:
$$
H = -\begin{pmatrix}
1 &0&0&0&0&0
\end{pmatrix}
(M_0 M_1 M_2)(M_0 M_1 M_2)(M_0 M_1 M_2)
\begin{pmatrix}
0 \\0\\0\\0\\0\\1
\end{pmatrix}.
$$
To put $Jx, Jy$ back in, we can replace $\sigma^x, \sigma^y$ with $\sqrt{J_x}\sigma^x, \sqrt{J_y}\sigma^y$.

Now we can implement these matrices in code.

In [1]:
import numpy as np

Jx = 0.4; Jy = 1.3
Sx = np.sqrt(Jx) * np.array([
    [0, 1],
    [1, 0]
])
Sy = np.sqrt(Jy) * np.array([
    [0, -1j],
    [1j, 0]
])
I = np.identity(2)
O = np.zeros_like(I)

M0 = np.array([
    [I, O, O, Sx, Sy, O],
    [O, I, O, O, O, O],
    [O, O, I, O, O, O],
    [O, O, O, O, O, Sx],
    [O, O, O, O, O, O],
    [O, O, O, O, O, I],
])
M1 = np.array([
    [I, O, Sx, O, Sy, O],
    [O, I, O, O, O, O],
    [O, O, O, O, O, Sx],
    [O, O, O, I, O, O],
    [O, O, O, O, I, Sy],
    [O, O, O, O, O, I],
])
M2 = np.array([
    [I, Sx, O, O, O, O],
    [O, O, O, O, O, Sx],
    [O, O, I, O, O, O],
    [O, O, O, I, O, O],
    [O, O, O, O, O, Sy],
    [O, O, O, O, O, I],
])

In contraction, there are two different types of multiplication: matrix multiplication and tensor product.

Thus we shall implement our own contraction function.

In [2]:
def bicontract(A, B):
    '''
    Contract two MPOs.
    '''
    assert(A.shape[1]==B.shape[0])
    res = np.zeros((A.shape[0], B.shape[1], A.shape[-1]*B.shape[-1], A.shape[-1]*B.shape[-1]), dtype=np.complex128)
    for row in range(A.shape[0]):
        for col in range(B.shape[1]):
            for ind in range(A.shape[1]):
                res[row, col] += np.kron(A[row, ind], B[ind, col])
    return res

def contract(op_list, power=1):
    '''
    Contract op_list to power.
    '''
    if power > 1:
        return bicontract(contract(op_list, power - 1), contract(op_list, 1))
    elif power == 1:
        if len(op_list) > 1:
            return bicontract(contract([op_list[0]], power), contract(op_list[1:], power))
        elif len(op_list) == 1:
            return op_list[0]
    return None

Now we can construct the Hamiltonian via contraction.

In [3]:
H = - contract([M0, M1, M2], 3)[0, -1]
print(H.shape)

(512, 512)


Indeed the Hamiltonian is $2^9 \times 2^9$.

## Diagonalization
Now, we diagonalize the Hamiltonian and print the lowest 20 eigenvalues.

In [4]:
eigval = np.linalg.eigvalsh(H)
print(eigval[:20])

[-11.79980076 -11.79980076 -11.79266065 -11.79266065 -11.79266065
 -11.79266065 -11.78554884 -11.78554884  -7.27512586  -7.27512586
  -7.27512586  -7.27512586  -7.27512586  -7.27512586  -7.19125783
  -7.19125783  -7.19125783  -7.19125783  -7.19125783  -7.19125783]


## Compare to Exact Diagonalization
To implement exact diagonalization, we encode basis vectors with binary strings and borrow bit operations from previous notebooks.

In [5]:
def readBit(num, n):
    return (num & (1 << n)) >> n

def flipBit(num, n):
    return num ^ (1 << n)

In [6]:
import numpy as np
from scipy import sparse

N = 9
length = 2 ** N

# Hamiltonian
HFrom = []
HTo = []
HValue = []

ijH = [(0,3), (3,6), (1,4), (4,7), (2,5), (5,8)]
ijV = [(0,1), (1,2), (2,0), (3,4), (4,5), (5,3), (6,7), (7,8), (8,6)]

for fromBasis in range(length):

    # horizontal
    for i, j in ijH:
        HFrom.append(fromBasis)
        HTo.append(flipBit(flipBit(fromBasis, i), j))
        HValue.append(-Jx)

    # vertical
    for i, j in ijV:
        HFrom.append(fromBasis)
        HTo.append(flipBit(flipBit(fromBasis, i), j))
        val = -1
        if readBit(fromBasis, i) == 1:
            val = -val
        if readBit(fromBasis, j) == 1:
            val = -val
        HValue.append(-Jy * val)

Hed = sparse.coo_matrix((HValue, (HTo, HFrom)), shape=(length, length)).toarray()

In [7]:
eigvaled = np.linalg.eigvalsh(Hed)
print(eigvaled[:20])

[-11.79980076 -11.79980076 -11.79266065 -11.79266065 -11.79266065
 -11.79266065 -11.78554884 -11.78554884  -7.27512586  -7.27512586
  -7.27512586  -7.27512586  -7.27512586  -7.27512586  -7.19125783
  -7.19125783  -7.19125783  -7.19125783  -7.19125783  -7.19125783]


Let's confirm that these two approaches yield the same result.

In [8]:
print('Hs are the same: ', H.all() == Hed.all())
print('Eigenvalues are the same:', (np.abs(eigval - eigvaled) < 1e-10).all())

Hs are the same:  True
Eigenvalues are the same: True


The Hamiltonians and eigen energies obtained from MPO and exact diagonlization are exactly the same!